# **Tiempo en el mercado**

In [18]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    if nombre.startswith("df_") and "alfa" in nombre and "pachuca" in nombre and 'agosto' in nombre
]
# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_agosto_2024_pachuca']

In [19]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'publicado_hace':'tiempo_de_publicacion',}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [20]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','tiempo_de_publicacion']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [7]:
nombres_df_filtrados

['df_alfa_abril_2024_pachuca',
 'df_alfa_agosto_2024_pachuca',
 'df_alfa_febrero_2024_pachuca',
 'df_alfa_julio_2024_pachuca',
 'df_alfa_junio_2024_pachuca',
 'df_alfa_marzo_2024_pachuca',
 'df_alfa_mayo_2024_pachuca',
 'df_alfa_q_feb_2023_pachuca',
 'df_alfa_q_jul_2023_pachuca',
 'df_alfa_q_jun_2023_pachuca',
 'df_alfa_q_mar_2023_pachuca',
 'df_alfa_q_nov_2022_pachuca',
 'df_alfa_q_oct_2022_pachuca',
 'df_alfa_q_sep_2023_pachuca']

In [21]:
dataframes_name = [
'df_alfa_agosto_2024_pachuca'
]

In [17]:
df_alfa_agosto_2024_pachuca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1406 entries, 0 to 1405
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     1406 non-null   int64  
 1   Categoria              1406 non-null   object 
 2   Precio                 1406 non-null   int64  
 3   propiedad              1406 non-null   object 
 4   metros_total           1406 non-null   int64  
 5   metros_construido      1406 non-null   int64  
 6   precio_m2_terreno      1406 non-null   float64
 7   precio_m2_construido   1406 non-null   float64
 8   tiempo_de_publicacion  1406 non-null   int64  
 9   Estado                 1406 non-null   object 
 10  Tipo                   1406 non-null   object 
 11  Estacionamientos       1406 non-null   int64  
 12  Recamaras              1406 non-null   int64  
 13  Banos                  1406 non-null   int64  
 14  Medio_banos            1406 non-null   int64  
 15  Bano

In [10]:
df_alfa_agosto_2024_pachuca['tiempo_de_publicacion'].value_counts().

tiempo_de_publicacion
7     277
3     161
2     126
16    126
1     112
4     100
6      87
5      76
10     73
9      59
8      39
11     31
14     29
15     23
21     21
13     19
12     17
17     16
18      7
22      4
19      3
Name: count, dtype: int64

# 1. Tiempo en el mercado

In [22]:
# Definir la función de categorización
def categorizar_en_meses(valor):
    if  valor <= 1:  # 0-1 mes
        return '0-30'  # Aproximadamente 0-30 días
    elif 2 <= valor <= 4:  # 2-4 meses
        return '31-120'  # Aproximadamente 31-120 días
    elif 5 <= valor <= 8:  # 5-8 meses
        return '121-240'  # Aproximadamente 121-240 días
    elif 9 <= valor <= 12:  # 9-12 meses
        return '241-364'  # Aproximadamente 241-364 días
    else:  # Más de 12 meses
        return '>=365'  # Más de 365 días
    

    #if isinstance(valor, int):  # Verificar si el valor es un número entero (meses)
        

# Definir el orden de las categorías
todas_categorias = ['0-30', '31-120', '121-240', '241-364', '>=365']

# Diccionario para almacenar los DataFrames de conteo por nombre de DataFrame original
dataframes_conteo_por_nombre = {}

# Lista para almacenar los DataFrames de conteo
lista_dataframes_conteo = []

# Iterar sobre cada DataFrame
for name, df in zip(dataframes_name, dataframes_list):
    # Aplicar la función de categorización
    df = df.copy()
    df['dias'] = df['tiempo_de_publicacion'].apply(categorizar_en_meses)
    print(df['dias'].value_counts())
    # Contar el número de registros únicos por cada valor único en la columna 'dias'
    conteo_por_dias = df['dias'].value_counts().reindex(todas_categorias, fill_value=0)
    # Crear un nuevo DataFrame de conteo
    df_conteo = pd.DataFrame({'dias': conteo_por_dias.index, 'conteo': conteo_por_dias.values})
    # Ordenar el DataFrame según el orden definido en todas_categorias
    df_conteo['dias'] = pd.Categorical(df_conteo['dias'], categories=todas_categorias, ordered=True)
    df_conteo = df_conteo.sort_values(by='dias')

 # Almacenar el DataFrame de conteo en la lista
    lista_dataframes_conteo.append(df_conteo)

# Concatenar todos los DataFrames de la lista
df_concatenado = pd.concat(lista_dataframes_conteo)

# Agrupar por categoría ('dias') y sumar los conteos
df_final = df_concatenado.groupby('dias').sum().reset_index()

# Imprimir el DataFrame final con las sumas por categoría
print(df_final)

dias
121-240    479
31-120     387
>=365      248
241-364    180
0-30       112
Name: count, dtype: int64
      dias  conteo
0     0-30     112
1   31-120     387
2  121-240     479
3  241-364     180
4    >=365     248


C:\Users\yoe11\AppData\Local\Temp\ipykernel_16260\2398907460.py:48: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [23]:
values = df_final['conteo']
labels = df_final['dias']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=labels, 
    y=values, 
    marker_color=colores,  # Especifica los colores de las barras
    text=values,  # Texto que se mostrará en las barras 
    textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
))
fig.update_layout(
    #title='Número de unidades por días en el mercado',  # Título de la gráfica
    xaxis=dict(title='Rango días'),  # Título del eje x
    yaxis=dict(
        title='Propiedades', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
         gridwidth=1,
             ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)    
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_tiempo_mercado', carpeta='assets\graficas')
fig.show()

### Directos

In [3]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica
import plotly.express as px
import pandas as pd
import os
import plotly.io as pio

data = {
    'dias':['0-10 días','11-30 días','31-100 días','101-365 días','>366 días' ],
    'conteo':[318,233,254,429,385],
}

# Convertir a DataFrame
df_final = pd.DataFrame(data)


In [4]:
values = df_final['conteo']
labels = df_final['dias']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=labels, 
    y=values, 
    marker_color=colores,  # Especifica los colores de las barras
    text=values,  # Texto que se mostrará en las barras 
    textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
))
fig.update_layout(
    #title='Número de unidades por días en el mercado',  # Título de la gráfica
    xaxis=dict(title='Rango días'),  # Título del eje x
    yaxis=dict(
        title='Propiedades', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
         gridwidth=1,
             ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)    
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)
guardar_grafico_como_html(fig, 'g_bar_tiempo_mercado', carpeta='assets\graficas')
fig.show()

<>:24: SyntaxWarning: invalid escape sequence '\g'
<>:41: SyntaxWarning: invalid escape sequence '\g'
<>:24: SyntaxWarning: invalid escape sequence '\g'
<>:41: SyntaxWarning: invalid escape sequence '\g'
C:\Users\yoe11\AppData\Local\Temp\ipykernel_17732\236012541.py:24: SyntaxWarning: invalid escape sequence '\g'
  def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
C:\Users\yoe11\AppData\Local\Temp\ipykernel_17732\236012541.py:41: SyntaxWarning: invalid escape sequence '\g'
  guardar_grafico_como_html(fig, 'g_bar_tiempo_mercado', carpeta='assets\graficas')


# 2. Riesgo

In [24]:
# Definir los valores para la columna 'stock'
stock_values = []
for dias in df_final['dias']:
    if dias == '>=365':
        stock_values.append('Más de un año en stock')
    else:
        stock_values.append('Menos de un año en stock')

# Crear el DataFrame 'riesgo'
riesgo = pd.DataFrame({
    'stock': stock_values,
    'casas': df_final['conteo']
})

# Agrupar por la columna 'stock' y sumar los valores de 'casas'
riesgo_stock = riesgo.groupby('stock').sum().reset_index()
# Ordenar el DataFrame para que 'Menos de un año en stock' quede abajo
riesgo_stock = riesgo_stock.sort_values(by='stock', ascending=False)
riesgo_stock

,stock,casas
1,Más de un año en stock,248
0,Menos de un año en stock,1158


In [25]:
labels = riesgo_stock['stock']
values = riesgo_stock['casas']
colores = ['#9500ff', '#ff0059']
# Crear el gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=labels, 
    values=values, 
    sort=False,    
    marker=dict(colors=colores,)
)])
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10),)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_pie_riesgo', carpeta='assets\graficas')
fig.show()

## DIRECTOS

In [5]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

data = {
    'stock': ['Más de un año en stock','Menos de un año en stock'],
    'casas': [814,551],    
}
# Convertir a DataFrame
riesgo_stock = pd.DataFrame(data)

In [6]:
labels = riesgo_stock['stock']
values = riesgo_stock['casas']
colores = ['#9500ff', '#ff0059']
# Crear el gráfico de pastel sin mostrar la leyenda
fig = go.Figure(data=[go.Pie(
    labels=labels, 
    values=values, 
    sort=False,    
    marker=dict(colors=colores),
    showlegend=False  # Ocultar la leyenda
)])
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10),)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)
guardar_grafico_como_html(fig, 'g_pie_riesgo', carpeta='assets\graficas')
fig.show()

<>:14: SyntaxWarning:

invalid escape sequence '\g'

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:14: SyntaxWarning:

invalid escape sequence '\g'

<>:31: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3084\2029931282.py:14: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3084\2029931282.py:31: SyntaxWarning:

invalid escape sequence '\g'

